In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, coupled_flux_qubit_non_linear_approx_pot_break_down, coupled_flux_qubit_non_linear_approx_force_break_down, coupled_flux_qubit_non_linear_approx_pot_only_d_beta
import edward_tools.coupled_fq_potential as coupled_fq_potential_package

import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit, separate_by_state
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from PARAMETER_INPUT import mapping_state_1_to_state_2_dict
# from PARAMETER_INPUT import *
# from PARAMETER_INPUT import _lambda, _theta, _eta
from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time, getFidelityTimeGraph
# from edward_tools.Analysis_tool.minimum_value_of_potential import animate_sim_flux_qubit_with_cutline
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
import kyle_tools as kt
import matplotlib.pyplot as plt

import importlib, os, datetime
from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library
import edward_tools.cfq_batch_sweep as cfq_batch_sweep



coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

In [ ]:
C_factor = 100
L_factor = 0.35
thermal_factor = 1

# R_factor = 0.1
R_factor = 1
I_m_factor = 0
# I_m_factor = 15
I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                                # Amp
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9 * L_factor, 1e-9 * L_factor                             # H 

R_1, R_2 = 371 * R_factor, 371 * R_factor                                # ohm 

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
# time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2
U0_2 = m_2 * x_c**2 / t_c**2
U0_1_k_BT_ratio = U0_1 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = k_BT/U0_1, k_BT/U0_1, k_BT/U0_1, k_BT/U0_1 # these are kappa' in the equation

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;



_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4]) *thermal_factor
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4]) *thermal_factor

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]

params['target_work'] = None
params['comment'] = "test new save method"
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)



In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
# phi_1_dcx, phi_2_dcx = 3, 3
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_x_off = 0
phi_2_x_off = 0
phi_1_dcx_off = 0
phi_2_dcx_off = 0
M_12_off = 0

# bookmark
phi_1_x_on = -0.193605263157894
phi_2_x_on = -0.193605263157894

phi_1_x_on = 0
phi_2_x_on = 0
# phi_1_x_on = -d_beta_1 * np.sin(1.556) / 1.55
# phi_2_x_on = -d_beta_2 * np.sin(1.556) / 1.55

# print(d_beta_2 * np.sin(1.556) / 1.556)

phi_1_dcx_on = 3.0
phi_2_dcx_on = 3.0
M_12_on = -0.6

initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}


# initial_parameter_dict = {
#         "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
#         "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
#         "phi_1_x": -0.10,  "phi_2_x": -0.10,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
#         "M_12": M_12_off, 'x_c': x_c
# }




In [ ]:
zeroDissipation = False
saveAllStates = True
_lambda_multiplier = 1000
params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "phi_1_x_on": phi_1_x_on, "phi_2_x_on": phi_1_x_on,
    "phi_1_dcx_on": phi_1_dcx_on, "phi_2_dcx_on": phi_2_dcx_on, "M_12_on": M_12_on,
    "gamma": gamma
}

params['sim_params'] = [_lambda, _theta, _eta]
params['sim_params'] = [_lambda * _lambda_multiplier, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off)]

protocol_index = 0

# bookmark
# initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = TR_initial_condition[protocol_index-1]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
TR_initial_condition[0]






protocol_list = [
    # forward
    # {"duration":50, "phi_2_x": phi_2_x_on * 0.5,  "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    {"duration":200,"phi_1_x": -1.5, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    # {"duration":100, "phi_2_x": phi_2_x_off, "phi_1_dcx": phi_1_dcx_off,  "M_12": M_12_off,   "name":"(2) conditional tilt in y"},

#     {"duration":200, "phi_2_x": phi_2_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"return to 4 well"},
    
#     {"duration":60, "phi_1_x": phi_1_x_on, "phi_2_x": phi_2_x_off, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
#     {"duration":60, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
#     {"duration":200, "phi_1_x": phi_1_x_on * 0.5, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
#     {"duration":50,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
#     {"duration":200,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
]

time_scale = 1
# protocol_list = [
#     # forward
#     {"duration":50 * time_scale, "phi_2_x": phi_2_x_on * 0.2,  "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name": r"Substage 1: Mix along $\varphi_2$ direction (1)"},
#     {"duration":200 * time_scale,"phi_2_x": phi_2_x_on, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name": r"Substage 2: Mix along $\varphi_2$ direction (2)"},
#     {"duration":60 * time_scale, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name": r"Substage 3: Conditional Tilt along $\varphi_2$ direction"},
#     {"duration":200 * time_scale, "phi_2_x": phi_2_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"Substage 4: return to 4 well"},
#     {"duration":60 * time_scale, "phi_1_x": phi_1_x_on, "phi_2_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": r"Substage 5: Conditional Tilt along $\varphi_1$ direction"},
#     {"duration":60 * time_scale, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name": r"Substage 6: Mix along $\varphi_2$ direction (2)"},
#     {"duration":200 * time_scale, "phi_1_x": phi_1_x_on * 0.2, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"Substage 7: return to 4 well (1)"},
#     {"duration":50 * time_scale,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name": "Substage 8: return to 4 well (2)"},
# ]


# protocol_list = [
#     # forward
#     {"duration":50 * time_scale,  "phi_1_dcx": phi_1_dcx_off,      "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name": r"Substage 1: Mix along $\varphi_2$ direction (1)"},
#     {"duration":200 * time_scale, "phi_1_dcx": phi_1_dcx_off,      "phi_2_dcx": phi_2_dcx_on,        "M_12": M_12_off,  "name": r"Substage 2: Mix along $\varphi_2$ direction (2)"},
#     {"duration":60 * time_scale,  "phi_1_dcx": phi_1_dcx_off,      "phi_2_dcx": phi_2_dcx_on,        "M_12": M_12_on,   "name": r"Substage 3: Conditional Tilt along $\varphi_2$ direction"},
#     {"duration":200 * time_scale, "phi_1_dcx": phi_1_dcx_off,      "phi_2_dcx": phi_2_dcx_off,       "M_12": M_12_off,  "name":"Substage 4: return to 4 well"},
#     {"duration":60 * time_scale,  "phi_1_dcx": phi_1_dcx_on,       "phi_2_dcx": phi_2_dcx_off,       "M_12": M_12_on,  "name": r"Substage 5: Conditional Tilt along $\varphi_1$ direction"},
#     {"duration":60 * time_scale,  "phi_1_dcx": phi_1_dcx_on,       "phi_2_dcx": phi_2_dcx_off,       "M_12": M_12_off,  "name": r"Substage 6: Mix along $\varphi_2$ direction (2)"},
#     {"duration":200 * time_scale, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off,       "M_12": M_12_off,  "name":"Substage 7: return to 4 well (1)"},
#     {"duration":50 * time_scale,  "phi_1_dcx": phi_1_dcx_off,      "phi_2_dcx": phi_2_dcx_off,       "M_12": M_12_off,  "name": "Substage 8: return to 4 well (2)"},
# ]

protocol_list = [
    # forward
    {"duration":40, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    # {"duration":40, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": -0.25,   "name":"(2) conditional tilt"},
    # {"duration":100, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": -0.25,   "name":"(3) raise the barrier"},
    {"duration":40, "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(4) 4 well potential 1st time"}, 
    {"duration":50, "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(4) 4 well potential 1st time"}, 

]


# tilting potential

In [ ]:
from scipy import optimize

In [ ]:
def tilt_phi_1(t, result):
    beta_1, beta_2       = result[4], result[5]
    d_beta_1, d_beta_2   = result[6], result[7]
    _phi_1x, _phi_2x     = result[8], result[9]
    _phi_1x, _phi_2x     = 0, 0
    _phi_1dcx, _phi_2dcx = result[10], result[11]
    _M_12                = result[12]
    _phi_1dc, _phi_2dc   = _phi_1dcx, _phi_2dcx
    _xi = 1 / (1 - _M_12**2)
    # print("M_12: ", _M_12)
    def Fcn(_phi_1):
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u3_1 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u4_1 = -d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)
        
        return u1_1 + u3_1 + u4_1

    
        
    solution_set = [optimize.fmin(Fcn, _g, disp=False) for _g in [-2, 2]]
    solution_set = [sol[0] for sol in solution_set]
    # print("solution_set_before: ", solution_set)    
    phi_1_a, phi_1_b = abs(max(solution_set)), -abs(max(solution_set))
    solution_set = [phi_1_a, phi_1_b]
    # phi_1_a, phi_1_b = solution_set
    # print("solution_set_after: ", solution_set)
    
    # correct_phi_1x = 1/2 * (phi_1_a + phi_1_b) - 1/(phi_1_a - phi_1_b) * \
    #                  (d_beta_1 * (np.sin(phi_1_a) - np.sin(phi_1_b)) * np.sin(_phi_1dc/2) -\
    #                  beta_1 * np.cos(_phi_1dc/2) * (np.cos(phi_1_a) - np.cos(phi_1_b)))
    phi_1_sol_set.append(solution_set)
    correct_phi_1x = 1/2 * (phi_1_a + phi_1_b) - 1/(phi_1_a - phi_1_b) * \
                 (d_beta_1 * (np.sin(phi_1_a) - np.sin(phi_1_b)) * np.sin(_phi_1dc/2) -\
                 beta_1 * np.cos(_phi_1dc/2) * (np.cos(phi_1_a) - np.cos(phi_1_b)))
    result[8] = correct_phi_1x
    
    return result


In [ ]:
def tilt_phi_2(t, result):
    beta_1, beta_2       = result[4], result[5]
    d_beta_1, d_beta_2   = result[6], result[7]
    _phi_1x, _phi_2x     = result[8], result[9]
    _phi_1x, _phi_2x     = 0, 0
    _phi_1dcx, _phi_2dcx = result[10], result[11]
    _M_12                = result[12]
    _phi_1dc, _phi_2dc   = _phi_1dcx, _phi_2dcx
    _xi = 1 / (1 - _M_12**2)

    
    def Fcn(_phi_2):
        u1_2 = 1/2 * _xi * (_phi_2 - _phi_2x)**2        
        u3_2 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u4_2 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)
        
        # u5 = _M_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)
        return u1_2 + u3_2 + u4_2


    solution_set = [optimize.fmin(Fcn, _g, disp=False) for _g in [-2, 2]]
    solution_set = [sol[0] for sol in solution_set]
    phi_2_a, phi_2_b = abs(min(solution_set)), -abs(min(solution_set))
    
    correct_phi_2x = 1/2 * (phi_2_a + phi_2_b) - 1/(phi_2_a - phi_2_b) * \
                 (d_beta_1 * (np.sin(phi_2_a) - np.sin(phi_2_b)) * np.sin(_phi_2dc/2) -\
                 beta_1 * np.cos(_phi_2dc/2) * (np.cos(phi_2_a) - np.cos(phi_2_b)))
    result[9] = correct_phi_2x

    return result

In [ ]:
solution_set_x_array = []
solution_set_y_array = []


def tilt_phi_1_2D(t, result):
    beta_1, beta_2       = result[4], result[5]
    d_beta_1, d_beta_2   = result[6], result[7]
    _phi_1x, _phi_2x     = result[8], result[9]
    _phi_1x, _phi_2x     = 0, 0
    _phi_1dcx, _phi_2dcx = result[10], result[11]
    _M_12                = result[12]
    _phi_1dc, _phi_2dc   = _phi_1dcx, _phi_2dcx
    _xi = 1 / (1 - _M_12**2)
    
    def Fcn(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u3_1 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u4_1 = d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)

        u1_2 = 1/2 * _xi * (_phi_2 - _phi_2x)**2        
        u3_2 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u4_2 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)

        u5 = _M_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)

        return u1_1 + u1_2 + u3_1 + u3_2 + u4_1 + u4_2 + u5

    solution_set = [optimize.fmin(Fcn, _g, disp=False) for _g in [[-2, -2], [2, 2]]]
    solution_set_x = [sol[0] for sol in solution_set]
    solution_set_y = [sol[1] for sol in solution_set]
    solution_set_x_array.append(solution_set_x)
    
    phi_1_a, phi_1_b = abs(min(solution_set_x)), -abs(min(solution_set_x))
    
    correct_phi_1x = 1/2 * (phi_1_a + phi_1_b) - 1/(phi_1_a - phi_1_b) * \
                 (d_beta_1 * (np.sin(phi_1_a) - np.sin(phi_1_b)) * np.sin(_phi_1dc/2) -\
                 beta_1 * np.cos(_phi_1dc/2) * (np.cos(phi_1_a) - np.cos(phi_1_b)))
    result[8] = correct_phi_1x
    
    return result

def tilt_phi_2_2D(t, result):
    beta_1, beta_2       = result[4], result[5]
    d_beta_1, d_beta_2   = result[6], result[7]
    _phi_1x, _phi_2x     = result[8], result[9]
    _phi_1x, _phi_2x     = 0, 0
    _phi_1dcx, _phi_2dcx = result[10], result[11]
    _M_12                = result[12]
    _phi_1dc, _phi_2dc   = _phi_1dcx, _phi_2dcx
    _xi = 1 / (1 - _M_12**2)
    
    def Fcn(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u3_1 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u4_1 = d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)

        u1_2 = 1/2 * _xi * (_phi_2 - _phi_2x)**2        
        u3_2 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u4_2 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)

        u5 = _M_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)

        return u1_1 + u1_2 + u3_1 + u3_2 + u4_1 + u4_2 + u5

    solution_set = [optimize.fmin(Fcn, _g, disp=False) for _g in [[-2, -2], [2, 2]]]
    solution_set_x = [sol[0] for sol in solution_set]
    solution_set_y = [sol[1] for sol in solution_set]
    phi_2_a, phi_2_b = abs(min(solution_set_y)), -abs(min(solution_set_y))
    solution_set_y_array.append(solution_set_y)
    
    correct_phi_2x = 1/2 * (phi_2_a + phi_2_b) - 1/(phi_2_a - phi_2_b) * \
                     (d_beta_2 * (np.sin(phi_2_a) - np.sin(phi_2_b)) * np.sin(_phi_2dc/2) -\
                     beta_2 * np.cos(_phi_2dc/2) * (np.cos(phi_2_a) - np.cos(phi_2_b)))
    result[9] = correct_phi_2x
    
    return result

In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""

# from edward_tools.Analysis_tool.minimum_value_of_potential import tilt_phi_1, tilt_phi_2

modifiedFunctionArray = [tilt_phi_2, tilt_phi_2, None, None, None, None, tilt_phi_1, tilt_phi_1]
modifiedFunctionArray = [tilt_phi_2_2D, tilt_phi_2_2D, None, None, None, None, tilt_phi_1_2D, tilt_phi_1_2D]
modifiedFunctionArray = [None, None, None, None, None, None, None, None]
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction=modifiedFunctionArray)

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""
cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, protocol_list = protocol_list)
protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(extra_constraint=extra_constraint_00_and_11_only)
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state




In [ ]:

_phi_1 = np.linspace(-4, 4, 1000)
line_colors = ["blue", "orange", "green", "red", "grey"]
phi_1_sol_set = [] 
test_t_array = np.linspace(630, 880, 5)
for _i, _t in enumerate(test_t_array):
    result = cfqr.protocol.get_params(_t)
    beta_1, beta_2       = result[4], result[5]
    d_beta_1, d_beta_2   = result[6], result[7]
    _phi_1x, _phi_2x     = result[8], result[9]
    _phi_1dcx, _phi_2dcx = result[10], result[11]
    _M_12                = result[12]
    _phi_1dc, _phi_2dc   = _phi_1dcx, _phi_2dcx
    _xi = 1 / (1 - _M_12**2)

    print("phi_1x:", _phi_1x)
    
    def U1_Fcn(_phi_1):
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u3_1 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u4_1 = -d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)
        
        return u1_1 + u3_1 + u4_1
    
    def cfq_potential_test(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u3_1 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u4_1 = d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)

        u1_2 = 1/2 * _xi * (_phi_2 - _phi_2x)**2        
        u3_2 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u4_2 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)

        u5 = _M_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)

        return u1_1 + u1_2 + u3_1 + u3_2 + u4_1 + u4_2 + u5

    
    plt.plot(_phi_1, cfq_potential_test([_phi_1, -1.96]))
    # plt.vlines(phi_1_sol_set[_i][0], ymax = 4, ymin = 0, colors = line_colors[_i])
    # plt.vlines(phi_1_sol_set[_i][1], ymax = 4, ymin = 0, colors = line_colors[_i])
    plt.ylim(0, 5)

In [ ]:
d_beta_1

In [ ]:
beta_1

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=[15, 11])
ax_flatten = ax.flatten()
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 0, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -1.96,\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                            showGraph=True)
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 200, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -1.96,\
                            contour_plt=ax_flatten[2], cutline_plt=ax_flatten[3], contours = 20,
                            showGraph=True)

# actual simulations

In [ ]:
# step 5: perform simulations
manual_domain=[np.array([-5, -5]), np.array([5, 5])]

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_saved, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                        percentage = 1, as_step = np.s_[::100], measure_all_states=saveAllStates, 
                        modifiedFunction=modifiedFunctionArray)

In [ ]:
simResult['cfqr'].getFidelityBarChart()

In [ ]:
simResult['cfqr'].getFidelityTimeGraph()

In [ ]:
# cutline graph

# , (1.96, "v","orange")
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
importlib.reload(coupled_fq_protocol_library)
ani,_,_ =  coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline(simResult['cfqr'], frame_skip = 10, time_array=simResult['cfqr'].sim.target_step_index, 
                                                                           params = params, cutlineInformation = [(-1.96, "v","red")], cutlineYLimit = [0, 250],
                                                                          pColor=pColor, opacity = 0, offset_potential = True)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
simResult['cfqr'].separate_by_state()

In [ ]:
state_00 = simResult['cfqr'].get_all_state()[simResult['cfqr'].particle_index['00']]
state_01 = simResult['cfqr'].get_all_state()[simResult['cfqr'].particle_index['01']]

In [ ]:
state_00_mean = np.mean(state_00, axis = 0)[:, 1, 0]
state_00_std = np.std(state_00, axis = 0)[:, 1, 0]
state_01_mean = np.mean(state_01, axis = 0)[:, 1, 0]
state_01_std = np.std(state_01, axis = 0)[:, 1, 0]

In [ ]:
sample_00 = np.random.normal(0, 0.15, 50000)

In [ ]:
# simResult['cfqr'].sim.target_step_index



In [ ]:
plt_line_data_at_t['cutline_plot']['targetU'] * 10 + 5000

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [3, 4])

from matplotlib.animation import FuncAnimation, PillowWriter
def animate(i):
    ax.clear()
    ax.set_ylim(0, 10000)
    ax.set_xlim(-3, 3)
    ax.get_yaxis().set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)

    ax.hist(sample_00 + state_00_mean[i], bins = 40, color = "red", alpha = 0.8)
    ax.hist(sample_00 + state_01_mean[i], bins = 40, color = "blue", alpha = 0.5)
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(simResult['cfqr'], t = simResult['cfqr'].sim.target_step_index[i] * params['dt'], cutlineDirection = "v")

    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -1.96,\
                        contour_plt=plt, cutline_plt=plt, contours = 20,
                        showGraph=False, cutlineColor = "red", cutlineRange = [-3, 3], cutlineYLimit = [0, 500], offset_potential = True)
    ax.plot(plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU'] * 100 + 5000, color = "black")

    
    
ani = FuncAnimation(fig, animate, interval=50, frames=len(state_00_mean), blit=False)


In [ ]:
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
# # no cutline graph

pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict, pColor = pColor, frame_skip= 20)

# fidelity analysis

In [ ]:
 fidelity_time_array[-1,:,1]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=[7, 7])

pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F7B6D2"}
fidelity_time_array = np.array(simResult['cfqr'].sim.fidelity_time_array)
# 00
ax[1][0].bar(["00", "01", "10", "11"], fidelity_time_array[-1,:,0].astype(float), color = pColor.values())
ax[1][0].set_ylim(0, 1)
ax[1][0].axhline(0.5, linestyle='--')

# 01
ax[0][0].bar(["00", "01", "10", "11"], fidelity_time_array[-1,:,1].astype(float), color = pColor.values())

ax[0][0].axhline(0.5, linestyle='--')
ax[0][0].set_ylim(0, 1)

# 10
ax[1][1].bar(["00", "01", "10", "11"], fidelity_time_array[-1,:,2].astype(float), color = pColor.values())
ax[1][1].set_ylim(0, 1)
ax[1][1].axhline(0.5, linestyle='--')

# 11
ax[0][1].bar(["00", "01", "10", "11"], fidelity_time_array[-1,:,3].astype(float), color = pColor.values())
ax[0][1].set_ylim(0, 1)
ax[0][1].axhline(0.5, linestyle='--')

# Work don analysis

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]

plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))
print(f"N = {params['N']}, dt = {params['dt']}")
print(f"phi_1dcx_on: {phi_1_dcx_on}, phi_2dcx_on: {phi_2_dcx_on}, M_12_on: {M_12_on}")
print(f"L_factor = {L_factor}, C_factor = {C_factor}, I_m_factor = {I_m_factor}")
print([x["duration"] for x in protocol_list])
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work}')

In [ ]:
# step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
step_array = simResult['cfqr'].sim.target_step_index
skip_step = int(len(step_array) * 0.05)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

plt.errorbar(step_array[::skip_step], work_mean[::skip_step], yerr = work_std[::skip_step])
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', linestyle ='--', label = 'axvline - full height')
    # plt.annotate(f"{i+1}", xy=(t, 4.5),  color='red', ha='center', size=16)

print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:

target_time_index = np.array(protocol_time_array)/params['dt']
target_time_index = target_time_index.astype("int") - 1

# work_done_array = [work_mean[i] for i in target_time_index[1:]]
work_done_array = simResult["cfqr"].sim.keyStep_work_statistic[:,0]
work_done_of_each_step = np.array(work_done_array)[1:] - np.array(work_done_array)[:-1]

Measure_Erasue_steps = [0, 1, 2, 3]
Control_Steps = [4, 5, 6, 7]

print("Third way to split the steps: (E, 0-1), (M. 2), (C: 3-6)")
print(f"work_done = {work_done_of_each_step}" )
print("measurement + reset = ", np.sum([work_done_of_each_step[i] for i in Measure_Erasue_steps]))
print("control = ", np.sum([work_done_of_each_step[i] for i in Control_Steps]))
print(sum(work_done_of_each_step))

In [ ]:
C_factor

# animations

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 5
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
# all_state = simResult["cfqr"].sim.output.all_state['states'][particle_number:particle_number+1, :, :, :]
all_state = simResult["cfqr"].sim.output.all_state['states']
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain, legend=False)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
ani,_,_ =  coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline(simResult['cfqr'], frame_skip = 5, time_array=simResult['cfqr'].sim.target_step_index, 
                                                                           params = params, cutlineInformation = [(-1.96, "v","red")])
# from IPython.display import HTML
# from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
importlib.reload(cfq_batch_sweep)

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery")

# plot shots

In [ ]:
minimum_point_data = [cfqr.system.get_minimum_point(_t, coupled_flux_qubit_non_linear_approx_pot, guess=[(-2, -2)]) for _t in protocol_time_array]

In [ ]:
from edward_tools.visualization import separate_by_state

In [ ]:
protocol_time_index_array

In [ ]:
all_state = simResult['cfqr'].sim.output.all_state['states']
all_state = all_state[:, :, (0, 1), :]
N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
x_array = [all_state[item] for item in which_axes]
state_lookup = separate_by_state(all_state)
opacity = min(1, 300/N)
x, y = x_array[0], x_array[1]
x_lim = (np.min(x), np.max(x))
y_lim = (np.min(y), np.max(y))
scat_kwargs = {'alpha':opacity, 'zorder':10}

In [ ]:
 protocol_time_array[_i]

In [ ]:
for _i in range(1, 5):
    print(list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1))
    # simResult['cfqr'].sim.target_step_index[np.argwhere(protocol_time_index_array==5000)[0][0]]

In [ ]:
protocol_time_index_array

In [ ]:
simResult['cfqr'].sim.target_step_index[0:30]

In [ ]:
simResult['cfqr'].sim.target_step_index[np.argwhere(protocol_time_index_array==4999)[0][0]]

In [ ]:
min_U_along_cutline = []
fig, ax = plt.subplots(5, 4, figsize=[15, 18])
ax_flatten = ax.flatten()
cutlineRange = [-3.5, 3.5]

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[0], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[0][0][0],\
                        contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                        showGraph=True, time=protocol_time_array[0], cutlineRange = cutlineRange)
min_U = np.min(U)
scat = [ax_flatten[2 * 0].scatter(x[state_lookup[key], protocol_time_index_array[0]], y[state_lookup[key], protocol_time_index_array[0]], c = pColor[key],**scat_kwargs) for key in state_lookup]


for _i in range(1, 5):
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "v")
    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][0],\
                            contour_plt=ax_flatten[2*_i], cutline_plt=ax_flatten[2*_i + 1], contours = 20,
                            showGraph=True, time=protocol_time_array[_i], cutlineRange = cutlineRange)
    min_U = np.min(U)
    _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
    scat = [ax_flatten[2 * _i].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
    
for _i in range(5, 9):
    X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "h")
    plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][1],\
                            contour_plt=ax_flatten[2*_i], cutline_plt=ax_flatten[2*_i + 1], contours = 20,
                            showGraph=True, time=protocol_time_array[_i], cutlineRange = cutlineRange)
    min_U = np.min(U)
    _index = list(simResult['cfqr'].sim.target_step_index).index(protocol_time_index_array[_i]+1)
    scat = [ax_flatten[2 * _i].scatter(x[state_lookup[key], _index], y[state_lookup[key], _index], c = pColor[key], **scat_kwargs) for key in state_lookup]
   

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=[6.5, 3])
ax_flatten = ax.flatten()
_i = 7

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[_i], cutlineDirection = "h")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_point_data[_i][0][0],\
                        contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                        showGraph=True,  time=protocol_time_array[_i])
min_U = np.min(U)
scat = [ax_flatten[0].scatter(x[state_lookup[key], protocol_time_index_array[_i]], y[state_lookup[key], protocol_time_index_array[_i]], **scat_kwargs) for key in state_lookup]

# minimum value of potential energy

In [ ]:
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
import importlib
importlib.reload(minimum_value_of_potential)

In [ ]:
minimum_value_of_potential.find_minimum_for_all_potential_2(simResult['cfqr'], 250)

In [ ]:
np.sin(1.9643321788037167)/1.9643321788037167 * d_beta_1

# KE analysis

In [ ]:
def get_all_state(self):
    return self.sim.output.all_state['states']

def get_time_step_indices(self):
    time_indices_array = np.array(self.sim.output.all_state['step_indices'])
    time_array = time_indices_array * cfqr.sim.dt
    return time_indices_array, time_array

In [ ]:
all_state = get_all_state(simResult['cfqr'])
time_indice_array, time_array = get_time_step_indices(simResult['cfqr'])


In [ ]:
KE_time_array = np.empty([cfqr.sim.ntrials, cfqr.sim.nsteps + 1])

for _i, _t in zip(time_step_indices, time_array):
    KE_time_array[:, _i] = simResult['cfqr'].system.get_kinetic_energy(all_state[:,_i, ...])
    

In [ ]:
all_state[:, :, 0, 1]**2 / 2 + all_state[:, :, 1, 1]**2 / 2  + all_state[:, :, 2, 1]**2 / 2 + all_state[:, :, 3, 1]**2 / 2 

In [ ]:
simResult['cfqr'].sim.output.all_state

# Trident_analysis

In [ ]:
def work_categories(work_dist, all_states, bins = [0.2, 1.5]):
    """
    This function will separate the particles by categories and from that find out the thing that distinguish them from each other.
    
    work_dist_left = the distribution on the left of the trident distribution
    """
    
    work_dist_left = all_states[work_dist < bins[0]]
    work_dist_middle = all_states[np.logical_and(work_dist >= bins[0], work_dist <= bins[1])]
    work_dist_right = all_states[work_dist > bins[1]]
    
    return work_dist_left, work_dist_middle, work_dist_right
                                  
    

In [ ]:
work_dist_left, work_dist_middle, work_dist_right = work_categories(simResult["work_distribution"], simResult['cfqr'].sim.output.all_state['states'])

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 5000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
# all_state = simResult["cfqr"].sim.output.all_state['states'][particle_number:particle_number+1, :, :, :]
all_state = work_dist_middle
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
all_state = work_dist_left
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
all_state = work_dist_right
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
all_state = simResult['cfqr'].sim.output.all_state['states']
plt.plot(all_state[::100, :, 1, 0])
plt.show()

In [ ]:


fig, ax = plt.subplots(1, 2, figsize = [12, 4])

for d in work_dist_left[::100]:
    ax[0].plot(d[:, 2, 0], "b")
    
for d in work_dist_right[::100]:
    ax[1].plot(d[:, 2, 0], "g")
    
for d in work_dist_middle[::100]:
    ax[0].plot(d[:, 2, 0], "r", alpha=0.1)
    ax[1].plot(d[:, 2, 0], "r", alpha=0.1)
    
    

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = [12, 4])

for d in work_dist_left[::100]:
    ax[0].plot(d[50000:, 3, 0], "b")
    
for d in work_dist_right[::100]:
    ax[1].plot(d[50000:, 3, 0], "g")
    
for d in work_dist_middle[::100]:
    ax[0].plot(d[50000:, 3, 0], "r", alpha=0.2)
    ax[1].plot(d[50000:, 3, 0], "r", alpha=0.2)
    
    

In [ ]:
plt.plot(np.mean(work_dist_left[:, :, 2, 0], axis = 0))
plt.plot(np.mean(work_dist_middle[:, :, 2, 0], axis = 0))
plt.plot(np.mean(work_dist_right[:, :, 2, 0], axis = 0))


In [ ]:
protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_all_time_index_array = cfqr.createProtocolTimeArray(protocol_list, params)

target_filter = np.logical_and(protocol_all_time_array >= 1000, protocol_all_time_array <= 1200)
target_time_array = protocol_all_time_array[target_filter]
target_time_index_array = protocol_all_time_index_array[target_filter]
work_dist_left_in_t_range = work_dist_left[:, target_filter, ...]
work_dist_middle_in_t_range = work_dist_middle[:, target_filter, ...]
work_dist_right_in_t_range = work_dist_right[:, target_filter, ...]

In [ ]:
from matplotlib.animation import FuncAnimation


def plot_work_dist_with_time(cfqr, work_dist, pot_function, target_time_array, target_time_index_array, skipNumber = 250):
    manual_domain=[np.array([-3, -3]), np.array([3, 3])]
    modified_manual_domain = [(manual_domain[0][1], manual_domain[0][0]), (manual_domain[1][1], manual_domain[1][0])]

    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(5,5)
    points = [(0.1, 0.5), (0.5, 0.5), (0.9, 0.5)]
    resolution = 50
    contours = 20
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(5,5)
    points = [(0.1, 0.5), (0.5, 0.5), (0.9, 0.5)]
    _skipped_target_time_array = target_time_array[::skipNumber]
    _skipped_target_time_index_array = target_time_index_array[::skipNumber]
    
    x_coordinate_of_dist_array = work_dist[:, :, 0, 0]
    U_of_dist_array = getEnergyTimeArrayOfDist(cfqr, work_dist, target_time_array, target_time_index_array, skipNumber)
    
    ax.scatter(x_coordinate_of_dist_array[0], U_of_dist_array[0])
    
    
    def animate(i):
        ax.clear()
        # Get the point from the points list at index i
        _t = _skipped_target_time_array[i]
        _params_at_t = cfqr.protocol.get_params(_t)
        beta_1 = _params_at_t[4]
        beta_2 = _params_at_t[5]
        d_beta_1 = _params_at_t[6]
        d_beta_2 = _params_at_t[7]
        _phi_1x = _params_at_t[8]
        _phi_2x = _params_at_t[9]
        _phi_1dcx = _params_at_t[10]
        _phi_2dcx = _params_at_t[11]
        _M_12 = _params_at_t[12]
        _xi = 1 / (1 - _M_12**2)
        slice_values = [0, 0, _phi_1dcx, _phi_2dcx]
        
        U, X_mesh = cfqr.system.lattice_with_self_defined_potential(_t, pot_function, resolution, axes=(0, 1),\
                                    manual_domain=modified_manual_domain, slice_values = slice_values)
        x_array, y_array = X_mesh
        
        selected_U = U[np.all(np.logical_and(y_array > 0.5, y_array < 3), axis = 1)]
        selected_x = x_array[np.all(np.logical_and(y_array > 0.5, y_array < 3), axis = 1)]

        ax.set_title(f"t = {_t}")
        for _x, _U in zip(selected_x, selected_U):
            ax.plot(_x, _U)
        ax.scatter(x_coordinate_of_dist_array[i], U_of_dist_array[i])
    
    ani = FuncAnimation(fig, animate, frames=len(_skipped_target_time_index_array),
                        interval=100, repeat=False)
    plt.close()
    return ani



In [ ]:
target_time_index_array

In [ ]:
ani = plot_work_dist_with_time(cfqr, work_dist_left, coupled_flux_qubit_non_linear_approx_pot,
                        target_time_array, target_time_index_array)

from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)



In [ ]:
def getEnergyTimeArrayOfDist(cfqr, work_dist, target_time_array, target_time_index_array, skipNumber = 1):
    U_of_dist_time_array = np.empty([work_dist.shape[0], len(target_time_index_array[::skipNumber])])
    
    for _j, _t_index in enumerate(target_time_index_array[::skipNumber]):
        _t = target_time_array[_t_index]
        U_of_dist_time_array[:, _j] =  cfqr.system.get_potential(work_dist[:, _t_index, ...], _t)
    return U_of_dist_time_array

In [ ]:
getEnergyTimeArrayOfDist(simResult['cfqr'], work_dist_left, protocol_all_time_array, protocol_all_time_index_array, skipNumber=10)

In [ ]:
KE_p1 = 1/2 * work_dist_right[0,:, 0, 1]**2

In [ ]:
all_state = work_dist_right[0:1,...]
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=1000, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)